In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
import os
import cv2
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, Layer, Dropout
from tensorflow.keras.models import Model

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import zipfile

file = zipfile.ZipFile('person.zip')

file.extractall()

file.close()

In [ ]:
def buat_dataset_siamese(path_images, labels, image_path):   
    #membuat list label yang tunggal
    unique_labels = np.unique(labels)

    #mencari index dari gambar dari suatu label
    tempat_gambar = {}
    for label in unique_labels:
        tempat_gambar[label] = [i for i, curr_label in enumerate(labels) if curr_label == label]

    #memasangkan images positif dan negatif    
    pair_images = []
    pair_labels = []
    for i, image in enumerate(image_path):
        image = cv2.imread(os.path.join(path_images, image), cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (200,200), interpolation=cv2.INTER_AREA)
        image = image/255.
        neg_index = [j for j, label in enumerate(labels) if label != labels[i]]
        post_index = tempat_gambar.get(labels[i])
    
        post_image = image_path[np.random.choice(post_index)]
        post_image = cv2.imread(os.path.join(path_images, post_image), cv2.IMREAD_GRAYSCALE)
        post_image = cv2.resize(post_image, (200,200), interpolation=cv2.INTER_AREA)
        post_image = post_image/255.
        pair_images.append((image, post_image))
        pair_labels.append(1)
    
        neg_image = image_path[np.random.choice(neg_index)]
        neg_image = cv2.imread(os.path.join(path_images, neg_image), cv2.IMREAD_GRAYSCALE)
        neg_image = cv2.resize(neg_image, (200,200), interpolation=cv2.INTER_AREA)
        neg_image = neg_image/255.
        pair_images.append((image, neg_image))
        pair_labels.append(0)
    images_dataset = np.array(pair_images, dtype='float64')
    labels_dataset = np.array(pair_labels, dtype='float64')

    images_dataset = np.expand_dims(images_dataset, axis=-1)

    images_dataset, labels_dataset = shuffle(images_dataset, labels_dataset)
    return images_dataset, labels_dataset

In [ ]:
path_images = os.path.join(os.getcwd(), 'person')
labels = [label.split('_')[0] for label in os.listdir(path_images) if label != "ipynb_checkpoints"]
image_path = os.listdir(path_images)
image_path.remove('.ipynb_checkpoints')
labels.remove('.ipynb')

In [ ]:
images_dataset, labels_dataset = buat_dataset_siamese(path_images, labels, image_path) 

In [ ]:
def buat_model_embedding():
    input = Input(shape=(200,200,1))
    
    #convolutional layers
    c = Conv2D(64, (10,10), activation='relu')(input)
    m = MaxPooling2D(64, (2,2), padding='same')(c)
    c = Conv2D(128, (7,7), activation='relu')(m)
    m = MaxPooling2D(128, (2,2), padding='same')(c)
    c = Conv2D(128, (7,7), activation='relu')(m)
    m = MaxPooling2D(128, (2,2), padding='same')(c)
    c = Conv2D(256, (4,4), activation='relu')(m)
    m = MaxPooling2D(256, (2,2), padding='same')(c)
    c = Conv2D(256, (4,4), activation='relu')(m)
    m = MaxPooling2D(256, (2,2), padding='same')(c)
    
    #Dense layers
    f = Flatten()(m)
    d = Dense(512, activation='relu')(f)
    d = Dense(1028, activation='relu')(d)
    d = Dense(128, activation='sigmoid')(d)
    
    return Model(inputs=[input], outputs=[d])

class distance(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    def call(self, anchor, validation):
        return tf.math.abs(anchor - validation)

In [ ]:
model = buat_model_embedding()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 191, 191, 64)      6464      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 96, 96, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 39, 39, 128)       802944

In [ ]:
def jadi_siamese():
    embedding = buat_model_embedding()
    jarak = distance()
    
    anchor_images = Input(shape=(200,200,1))
    validation_images = Input(shape=(200,200,1))
    
    anchor_embedding = embedding(anchor_images)
    validation_embedding = embedding(validation_images)
    
    kesamaan = jarak(anchor_embedding, validation_embedding)
    
    #dense layer
    d2 = Dense(1, activation='sigmoid')(kesamaan)
    
    return Model(inputs=[anchor_images, validation_images], outputs=[d2])

In [ ]:
siamese = jadi_siamese()
siamese.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 200, 200, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_4 (InputLayer)           [(None, 200, 200, 1  0           []                               
                                )]                                                                
                                                                                                  
 model_1 (Functional)           (None, 128)          4623556     ['input_3[0][0]',                
                                                                  'input_4[0][0]']          

In [ ]:
siamese.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])

history = siamese.fit([images_dataset[:,0,:], images_dataset[:,1,:]], labels_dataset, epochs=80, validation_split=0.2, batch_size=32, verbose=1)

Epoch 1/80
44/44 [==============================] - 160s 3s/step - loss: 0.6927 - accuracy: 0.5958 - val_loss: 0.6924 - val_accuracy: 0.7385
Epoch 2/80
44/44 [==============================] - 129s 3s/step - loss: 0.6922 - accuracy: 0.7111 - val_loss: 0.6919 - val_accuracy: 0.7385
Epoch 3/80
44/44 [==============================] - 129s 3s/step - loss: 0.6917 - accuracy: 0.7500 - val_loss: 0.6913 - val_accuracy: 0.7241
Epoch 4/80
44/44 [==============================] - 129s 3s/step - loss: 0.6910 - accuracy: 0.7522 - val_loss: 0.6906 - val_accuracy: 0.7299
Epoch 5/80
44/44 [==============================] - 129s 3s/step - loss: 0.6901 - accuracy: 0.7558 - val_loss: 0.6896 - val_accuracy: 0.7299
Epoch 6/80
44/44 [==============================] - 129s 3s/step - loss: 0.6889 - accuracy: 0.7673 - val_loss: 0.6880 - val_accuracy: 0.7241
Epoch 7/80
44/44 [==============================] - 129s 3s/step - loss: 0.6870 - accuracy: 0.7068 - val_loss: 0.6856 - val_accuracy: 0.7241
Epoch 8/80
44

In [ ]:
tes1 = cv2.imread('tes1.jpg', 0)
tes2 = cv2.imread('tes2.jpg', 0)
tes1 = cv2.resize(tes1, (200,200), interpolation=cv2.INTER_AREA)/255.
tes2 = cv2.resize(tes2, (200,200), interpolation=cv2.INTER_AREA)/255.

tes = (tes1, tes2)
tes = np.array([tes])
tes = np.expand_dims(tes, axis=-1)
tes.shape

(1, 2, 200, 200, 1)

In [ ]:
siamese.predict([tes[:,0,:], tes[:,1,:]])

1/1 [==============================] - 0s 144ms/step


array([[0.8596978]], dtype=float32)

In [ ]:
siamese.save('content/siamesemodel.h5')

In [ ]:
files = os.listdir('person')

for file in files:
  os.remove(os.path.join('person', file))

os.rmdir('person')